<a href="https://colab.research.google.com/github/alexlautw9527/for_notebook/blob/main/Fund2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 
import numpy as np

In [ ]:
fund_df = pd.read_csv('/content/drive/MyDrive/05_OTHER_DATA/基金產品列表.csv', encoding='CP950')

In [ ]:
fund_code_df = pd.read_csv('/content/drive/MyDrive/05_OTHER_DATA/FUND_CODE_DATA.csv', encoding='CP950')
fund_code_df = fund_code_df[['產品代碼', '基金名稱', 'ISIN Code']]

In [ ]:
fund_df = fund_df[fund_df['RISKLEVEL']!=0][['CNAME','FUNDCRYID', 'RISKLEVEL','DIVIDENDTYPE2','ASSETCAT_CODE','SECTOR_NAME']]
fund_df['SECTOR_NAME'] = fund_df['SECTOR_NAME'].fillna('NA')
fund_df = fund_df[~fund_df['ASSETCAT_CODE'].isna()]

In [ ]:
corpus = fund_df.astype(str).values.tolist()

In [ ]:
corpus[0:5]

[['施羅德２０２２到期新興市場主權債券基金（美元）－累積型（本基金進行配息前未先扣除應負擔之相關費用）（本基金有相當比重投資於非投資等級之高風險債券且配息來源可能為本金）',
  'USD',
  '3',
  '03_不配',
  'BOND',
  'NA'],
 ['施羅德２０２２到期新興市場主權債券基金（美元）－配息型（本基金進行配息前未先扣除應負擔之相關費用）（本基金有相當比重投資於非投資等級之高風險債券且配息來源可能為本金）',
  'USD',
  '3',
  '01_配息',
  'BOND',
  'NA'],
 ['新光創新科技基金（新臺幣）', 'TWD', '4', '03_不配', 'STOCK', '科技'],
 ['新光店頭基金（新臺幣）', 'TWD', '4', '03_不配', 'STOCK', '台灣股市'],
 ['新光台灣富貴基金（新臺幣）', 'TWD', '4', '03_不配', 'STOCK', '台灣股市']]

In [2]:
import pandas as pd
from gensim.models.word2vec import Word2Vec

In [ ]:
model = Word2Vec(corpus, size=50, iter=50,window=5, sg=1,negative=20,min_count=1)


In [ ]:
def most_similar(w2v_model, words, topn=20):
    similar_df = pd.DataFrame()
    for word in words:
        try:
            similar_words = pd.DataFrame(w2v_model.wv.most_similar(word, topn=topn), columns=[word, 'cos'])
            similar_df = pd.concat([similar_df, similar_words], axis=1)
        except:
            print(word, "not found in Word2Vec model!")
    return similar_df

In [ ]:
np.savetxt("fund_embedd.tsv", model.wv[fund_df['CNAME'].tolist()], delimiter="\t")


In [ ]:
fund_df.to_csv('fund_embedd_metadata.tsv', sep = '\t',index=False)

In [ ]:
fund_df.shape

(2650, 6)

In [ ]:
from google.colab import files
files.download('/content/fund_embedd_metadata.tsv') 
files.download('/content/fund_embedd.tsv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
! wget https://www.rahimikia.com/FinText/FinText_FastText_Skip-gram.zip

--2021-10-26 15:11:10--  https://www.rahimikia.com/FinText/FinText_FastText_Skip-gram.zip
Resolving www.rahimikia.com (www.rahimikia.com)... 69.163.228.90
Connecting to www.rahimikia.com (www.rahimikia.com)|69.163.228.90|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11656495869 (11G) [application/zip]
Saving to: ‘FinText_FastText_Skip-gram.zip’

FinText_FastText_Sk 100%[===================>]  10.86G  13.1MB/s    in 8m 13s  

2021-10-26 15:19:24 (22.5 MB/s) - ‘FinText_FastText_Skip-gram.zip’ saved [11656495869/11656495869]



In [5]:
!unzip /content/FinText_FastText_Skip-gram.zip

Archive:  /content/FinText_FastText_Skip-gram.zip
   creating: FinText_FastText_Skip-gram/
  inflating: FinText_FastText_Skip-gram/Word_Embedding_2000_2015  
  inflating: FinText_FastText_Skip-gram/README.txt  
  inflating: FinText_FastText_Skip-gram/Word_Embedding_2000_2015.trainables.vectors_ngrams_lockf.npy  
  inflating: FinText_FastText_Skip-gram/Word_Embedding_2000_2015.pkl  
  inflating: FinText_FastText_Skip-gram/Word_Embedding_2000_2015.wv.vectors_ngrams.npy  
  inflating: FinText_FastText_Skip-gram/Word_Embedding_2000_2015.wv.vectors.npy  
  inflating: FinText_FastText_Skip-gram/Word_Embedding_2000_2015.trainables.vectors_vocab_lockf.npy  
  inflating: FinText_FastText_Skip-gram/Word_Embedding_2000_2015.wv.vectors_vocab.npy  
  inflating: FinText_FastText_Skip-gram/Word_Embedding_2000_2015.trainables.syn1neg.npy  


In [6]:
from gensim.models import Word2Vec
from gensim.models import FastText
import gensim.downloader as api 

In [7]:
FinText_Word2Vec_skipgram = Word2Vec.load('/content/FinText_FastText_Skip-gram/Word_Embedding_2000_2015')

In [8]:

prod=['fund',
 'etf',
 'stock',
 'preferred_stock',
 'auto_loan',
 'trust',
 'life_insurance',
 'insurance',
 'personal_loan',
 'mortgage',
 'loan',
 'treasury_bond',
 'foreign_exchange',
 'online_banking',
 'fintech',
 'blockchain',
 'bitcoin',
 'robo_advisor'
 ]

In [9]:
product_embedding = FinText_Word2Vec_skipgram.wv[prod]

In [10]:
product_embedding

array([[ 0.14604235,  0.1704098 ,  0.01929267, ..., -0.13392453,
        -0.3286863 ,  0.01605617],
       [ 0.40752319,  0.3141233 , -0.22489968, ..., -0.28437993,
         0.02246348,  0.17138132],
       [ 0.09119733,  0.13523722,  0.01482047, ..., -0.09712761,
        -0.23065072, -0.22710954],
       ...,
       [-0.00303637,  0.145203  , -0.41418603, ..., -0.11681274,
        -0.14459747, -0.02398938],
       [-0.07647095,  0.24916057, -0.12600106, ...,  0.3001527 ,
        -0.21294053, -0.04931631],
       [ 0.29539302, -0.00322265,  0.12641694, ...,  0.13217312,
        -0.4011273 , -0.07066225]], dtype=float32)

In [11]:
product_embedding.shape

(18, 300)

In [12]:
np.savetxt("product_embedd.tsv", product_embedding, delimiter="\t")


In [13]:
pd.DataFrame({'product':prod}).to_csv('prod_embedd_metadata.tsv', sep = '\t',index=False,header=False)

In [14]:
from google.colab import files
files.download('/content/prod_embedd_metadata.tsv') 
files.download('/content/product_embedd.tsv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>